# SQL con clase

*The single biggest challenge to learning SQL programming is unlearning procedureal programming.*
-- Joe Celko

En este notebook, exploraremos diversos estilos y prácticas para escribir consultas SQL enfocadas en el análisis de datos. Las consultas analíticas se distinguen de las consultas convencionales en que están diseñadas específicamente para trabajar sobre grandes volúmenes de datos y tienen a ser más complejas.

## Setup

Primero vamos a configurar la notebook y definir las variables principales.

Instalamos los paquetes necesarios. Este paso se puede omitir si ya lo realizó en su entorno, sin embargo si usa **Colab** va a ser necesario ejecutarlo.

In [ ]:
%%bash
pip install duckdb
pip install jupysql
pip install duckdb-engine

Configuracion la extensión de %sql para el notebook, lo que nos va a permitir usar lenguaje SQL directamente sobre una celda

In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb://

Y finalmente cargamos el dataset en la base de datos

In [2]:
dataset_path = '/home/augusto/Desktop/dmeyf2025/Data/' # '/content/drive/MyDrive/DMEyF/2024/datos/'
dataset_file = 'competencia_01_crudo.csv'

In [3]:
%%sql
create or replace table competencia_01_crudo as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

,Success


## WITH



Un escenario común es necesitar realizar una consulta a no a una tabla, sino a otra consulta. Una forma tradicional de hacerlo es usando es:
* Creando una tabla temporal con esa consulta
* Usando `selects from select`.

Esta necesidad es muy común, pero como todo en la vida hay peros.
- Crear una tabla en una base de datos que va a ser usada UNA SOLA VEZ ensucia los esquemas con un montón de tablas que no son útiles y además del desorden ocupan espacio innecesario
- El `select from select`, debe ser de las cosas peor vistas por los administradores de BBDD. Tienen sus razones técnicas, en general la recomendación es no usarlo a menos que uno conozca bien como funciona la base de datos por dentro, ya que hay casos donde terminan ocasionando muchos problemas

Y que hacer? bueno hay una clausula en **SQL** llamada `WITH`, que nos permite crear esas (plural) tablas para ser usadas en la **query** principal. Ejemplo si queremos comparar el valor de una variable con su promedio a través de todos los meses.


In [4]:
%%sql
with promedios as (
    select
        numero_de_cliente
        , avg(mrentabilidad) as avg_mrentabilidad
    from competencia_01_crudo
    group by numero_de_cliente
) select
    cp.numero_de_cliente
    , cp.foto_mes
    , if(cp.mrentabilidad > p.avg_mrentabilidad, 'mayor', 'menor') as comparacion
from competencia_01_crudo cp
join promedios p using (numero_de_cliente)


,numero_de_cliente,foto_mes,comparacion
0,627884018,202102,mayor
1,627887374,202102,menor
2,627894889,202102,menor
3,627899996,202102,mayor
4,627907212,202102,menor
...,...,...,...
978434,935013868,202106,menor
978435,935015260,202106,mayor
978436,935015619,202106,menor
978437,935018172,202106,mayor


- Usar esa clausula tiene muchos beneficios, entre ellos sumar legibilidad de código y le permite al motor de **SQL** optimizar mejor, ya que entiende todo lo que se buscar hacer y planifica como resolverlo en el menor tiempo posible.

**NOTA**: Hay un tipo de tabla que se llaman temporales, que se borran solas una vez que uno se desconecta de la base de datos. Es una mejor alternativa frente al `CREATE`, sin embargo si la tabla que se esta creando se usa una sola vez es conveniente usar el `WITH` porque permite optimizar la ejecución. Si esa tabla va a ser usada mas de una vez dentro del proceso, las tablas temporales son la mejor opción.

# Funciones Analíticas

Las funciones analíticas en SQL son un conjunto de funciones que te permiten realizar cálculos avanzados sobre un conjunto de filas relacionadas dentro de una consulta conservando las filas individuales y calculando sobre ventanas de datos relacionadas.

Veamos un ejemplo para que quede más claro:

In [5]:
%%sql
select
    numero_de_cliente
    , foto_mes
    , mrentabilidad
from competencia_01_crudo
where numero_de_cliente = 918147740
order by foto_mes

,numero_de_cliente,foto_mes,mrentabilidad
0,918147740,202101,276.74
1,918147740,202102,-735.18
2,918147740,202103,-417.20
3,918147740,202104,-694.08
4,918147740,202105,-1692.76
5,918147740,202106,-897.20


In [6]:
%%sql
select
    numero_de_cliente
    , foto_mes
    , mrentabilidad
    , lead(mrentabilidad, 1) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_mas_1
    , lag(mrentabilidad, 1) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_menos_1
    , lead(mrentabilidad, 2) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_mas_2
    , lag(mrentabilidad, 2) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_menos_2
from competencia_01_crudo
where numero_de_cliente = 918147740
order by foto_mes



,numero_de_cliente,foto_mes,mrentabilidad,mrentabilidad_mas_1,mrentabilidad_menos_1,mrentabilidad_mas_2,mrentabilidad_menos_2
0,918147740,202101,276.74,-735.18,NaN,-417.20,NaN
1,918147740,202102,-735.18,-417.20,276.74,-694.08,NaN
2,918147740,202103,-417.20,-694.08,-735.18,-1692.76,276.74
3,918147740,202104,-694.08,-1692.76,-417.20,-897.20,-735.18
4,918147740,202105,-1692.76,-897.20,-694.08,NaN,-417.20
5,918147740,202106,-897.20,NaN,-1692.76,NaN,-694.08


* Qué paso?
* ¿Cómo esta construyendo las nuevas variables?
* ¿ Nos puede ayudar a construir targets ?

¿Dondé puedo leer un poco más de esta magia negra? https://duckdb.org/docs/sql/window_functions.html

## Los casos raros

Que pasa con el cliente **930544172** y la foto **202103**


In [7]:
%%sql
select
    numero_de_cliente
    , foto_mes
from competencia_01_crudo
where numero_de_cliente = 930544172
order by foto_mes

,numero_de_cliente,foto_mes
0,930544172,202101
1,930544172,202102


* ¿Cuál debería ser la clase para cada periodo?

### Una pista

Podemos generar todas las posibles combinaciones de clientes y periodos de manera muy simple

In [8]:
%%sql
with periodos as (
  select distinct foto_mes from competencia_01_crudo
), clientes as (
  select distinct numero_de_cliente from competencia_01_crudo
)
select numero_de_cliente, foto_mes from clientes cross join periodos
where numero_de_cliente = 930544172
order by foto_mes

,numero_de_cliente,foto_mes
0,930544172,202101
1,930544172,202102
2,930544172,202103
3,930544172,202104
4,930544172,202105
5,930544172,202106


Solo nos queda saber si estuvo o no el banco el cliente en ese periodo y armar el

In [9]:
%%sql
with periodos as (
    select distinct foto_mes from competencia_01_crudo -- Esto también se puede hacer con secuencias
), clientes as (
    select distinct numero_de_cliente from competencia_01_crudo
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
)
select
    t.numero_de_cliente
    , t.foto_mes
    , if(c.numero_de_cliente is null, 0, 1) as mes_0
from todo t
left join competencia_01_crudo c using (numero_de_cliente, foto_mes)
where t.numero_de_cliente = 930544172
order by foto_mes

,numero_de_cliente,foto_mes,mes_0
0,930544172,202101,1
1,930544172,202102,1
2,930544172,202103,0
3,930544172,202104,0
4,930544172,202105,0
5,930544172,202106,0


Antes de continuar, tomémonos un momento para reflexionar. Con todas estas piezas, ¿cómo podemos ensamblar el target?

Una vez que lo tengamos claro sobre el papel, proceda a completar el código que sigue.

In [10]:
%%sql
create or replace table clases_ternarias as with usuarios_ultimo_a_primer_mes as(  select foto_mes, numero_de_cliente, row_number() over (partition by numero_de_cliente order by foto_mes desc) as row_number from competencia_01_crudo) 
    select foto_mes,numero_de_cliente, 
    case
        when row_number = 1 and foto_mes < 202106 then 'BAJA+1'
        when row_number = 2 and foto_mes < 202105 then 'BAJA+2'
        when row_number >= 3 then 'CONTINUA'
    else null
end as clase_ternaria from usuarios_ultimo_a_primer_mes ;

,Success


In [11]:
%%sql
PIVOT clases_ternarias
ON clase_ternaria
USING COUNT(numero_de_cliente)
GROUP BY foto_mes
ORDER BY foto_mes;

,foto_mes,BAJA+1,BAJA+2,CONTINUA
0,202101,616,824,160087
1,202102,830,1027,160298
2,202103,1034,953,161122
3,202104,954,1131,161333
4,202105,1134,0,0
5,202106,0,0,0


Revisamos que todo salga como esperamos

In [12]:
%sql select count(*) from competencia_01_crudo

,count_star()
0,978439


Creamos las variables target y targetcluster

In [13]:
%%sql
CREATE or replace table clases_ternarias AS
SELECT
  *,
  CASE
    WHEN clase_ternaria = 'BAJA+2' THEN 1
    ELSE 0
  END AS target,
  CASE
    WHEN clase_ternaria IN ('BAJA+1', 'BAJA+2') THEN 1
    ELSE 0
  END AS targetcluster
FROM
  clases_ternarias;

,Success


In [14]:
%%sql
create or replace table competencia_01_curado as
SELECT crudo.*, clase.clase_ternaria, clase.target, clase.targetcluster FROM competencia_01_crudo AS crudo INNER JOIN clases_ternarias AS clase ON crudo.foto_mes = clase.foto_mes AND crudo.numero_de_cliente = clase.numero_de_cliente;

,Success


Y vemos la cardinalidad de las clases por periodo

Y finalmente almacenamos el nuevo dataset para ser usado a continuación

In [15]:
%sql COPY competencia_01_curado TO '{{dataset_path}}competencia_01.csv' (FORMAT CSV, HEADER)

,Success


In [16]:
%%sql
select * from competencia_01_curado

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria,target,targetcluster
0,249221323,202101,1,0,0,46,93,3070.24,15691.10,1278.11,...,-20833.47,0.0,2814,6743.38,6,0,9559.95,CONTINUA,0,0
1,249227600,202101,1,0,0,42,278,496.21,15321.80,418.97,...,-9662.51,0.0,5541,9419.90,8,0,1900.26,CONTINUA,0,0
2,249234235,202101,1,0,0,54,17,806.56,10549.80,174.26,...,-4692.00,0.0,577,2821.32,2,0,6885.51,CONTINUA,0,0
3,249244449,202101,1,0,0,61,290,27485.71,65621.74,23590.68,...,-20349.22,0.0,7916,23509.71,8,0,1219.92,CONTINUA,0,0
4,249244739,202101,1,0,0,66,320,3306.82,48787.99,3274.95,...,-88178.37,0.0,1958,20397.54,8,0,2627.52,CONTINUA,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978434,1598368433,202106,1,0,0,58,1,4123.09,4123.09,2242.51,...,0.00,0.0,28,12521.76,2,0,1747.77,None,0,0
978435,1598419415,202106,1,0,0,30,1,-2.26,-2.26,0.00,...,NaN,NaN,9,NaN,<NA>,<NA>,0.00,None,0,0
978436,1598425905,202106,1,0,0,28,1,5.92,5.92,0.00,...,NaN,NaN,20,NaN,<NA>,<NA>,0.00,None,0,0
978437,1598470389,202106,1,0,0,78,1,244.96,244.96,0.00,...,NaN,NaN,15,NaN,<NA>,<NA>,0.00,None,0,0


In [17]:
%%sql
PIVOT competencia_01_curado
ON target
USING COUNT(numero_de_cliente)
GROUP BY foto_mes
ORDER BY foto_mes;

,foto_mes,0,1
0,202101,160703,824
1,202102,161128,1027
2,202103,162156,953
3,202104,162287,1131
4,202105,163917,0
5,202106,164313,0


In [18]:
%%sql
PIVOT competencia_01_curado
ON targetcluster
USING COUNT(numero_de_cliente)
GROUP BY foto_mes
ORDER BY foto_mes;

,foto_mes,0,1
0,202101,160087,1440
1,202102,160298,1857
2,202103,161122,1987
3,202104,161333,2085
4,202105,162783,1134
5,202106,164313,0



## El pato del amor

DuckDB no es una base de datos SQL estándar; su sintaxis moderna facilita significativamente la escritura de consultas. A continuación, te menciono algunas características destacadas:

+ Uso de variables: En DuckDB, no es necesario reescribir las variables; puedes utilizar tus variables una vez definidas, lo que simplifica el código.

+ Función IF: DuckDB soporta la función IF(condición, valor_si_verdadero, valor_si_falso), lo que hace que tu código sea más claro y legible en comparación con el uso de CASE WHEN.

+ Exclusión de campos: Puedes utilizar SELECT * EXCLUDE(field1, ...) para excluir campos específicos de la lista de selección, lo que ahorra tiempo y esfuerzo.

+ Alias en cláusulas: Es posible utilizar alias en las cláusulas WHERE, GROUP BY y HAVING, lo que añade flexibilidad a la escritura de consultas.

+ Uso de alias en cálculos: DuckDB permite el uso de alias dentro del cálculo de otros campos, facilitando la manipulación y transformación de datos en una consulta.

Estas son solo algunas de las capacidades que hacen de DuckDB una herramienta poderosa. Te recomiendo leer la documentación y el blog oficial para explorar más a fondo su potencial.